# Circulation Data Conversion

This document, similar to WIMConversion, is for converting the traffic circulation data into a useable pickled form

In [ ]:
import pandas as pd
import numpy as np
import h5py

In [2]:
def load_table_from_struct_circ(station, year) -> pd.DataFrame():
    
    file =  h5py.File('{}/{}_{}_struct.mat'.format(station,station, year), 'r')
    circ_struct = file['circ_struct']
    
    data = []
    for ref in circ_struct['table']['data'][:]:
        data.append(file[ref[0]].value)
    
    columns = []
    for column in circ_struct['columns'][:]:
        columns.append(file[column[0]].value.tobytes()[::2].decode())
    
    table_dict = {}
    for colidx in range(len(columns)):
        table_dict[columns[colidx]] = data[colidx][0]
        
    #For some reason there is this "c" column that is only of length 6, drop this
    del table_dict['c']
    return pd.DataFrame(table_dict)

In [ ]:
def circ_df_pickle(df):
    df = df.fillna(0)
    to_int = ['n', 'd', 'h', 'm', 's', 'ms', 'Lane', 'Dir', 'SwissT']
    df[to_int] = df[to_int].astype(int)
    #Less than 1% null, therefore remove the null values
    is_null = df.d.isnull()
    df = df[df.d != is_null]
    df['d_str'] = df.d.astype(str)
    df['len_d'] = df['d_str'].apply(lambda x: len(x))
    #Eliminate any date that does not have the minimum length of 5
    df = df[df.len_d.isin([5, 6])]
    df['d_str'] =df.apply(lambda x: x['d_str'] if x['len_d'] == 6 else '0' + x['d_str'], axis=1)
    df['Date'] = df['d_str'].apply(lambda x: pd.to_datetime(x, errors='coerce', format='%d%m%y'))
    df = df.drop(columns=['d'])
    df['Date'] += pd.to_timedelta(df.h, unit='h')
    df['Date'] += pd.to_timedelta(df.m, unit='m')
    df['Date'] += pd.to_timedelta(df.s, unit='s')
    df['Date'] += pd.to_timedelta(df.ms, unit='ms')
    df = df.drop(columns=['h', 'm', 's', 'ms', 'd_str', 'len_d'])
    df['index'] = df.index #Create new index, current index n resets itself
    df = df.drop(columns='n')
    df = df.dropna() #Drop any values that are still null (only 2 for 2017)
    lanes = range(1, 7)
    direc = [1, 2]
    df = df[df.Lane.isin(lanes)]
    df = df[df.Dir.isin(direc)]
    df = df[df.SwissT.isin(range(1, 11))] #Ensures only swiss10 classifications are kept
    pd.to_pickle(df, '{}/{}_circ.pkl'.format(station, year))


In [3]:
station = '137'
year = 2017
df = load_table_from_struct_circ(station, year)
circ_df_pickle()

C:\Users\Isabelle\Anaconda3\lib\site-packages\h5py\_hl\dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
